# DSAP 107

In [1]:
import numpy as np
import pandas as pd 
import datetime
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['figure.dpi'] = 130

In [16]:
def to_datetime(x):
    tempt = datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S %Z') + datetime.timedelta(hours = 8)
    return tempt.hour*100 + tempt.minute

def time_period(t):
    if t >= 2300 or t < 300:
        return 0
    elif t >= 300 and t < 700:
        return 1
    elif t >= 700 and t < 1100:
        return 2
    elif t >= 1100 and t < 1500:
        return 3
    elif t >= 1500 and t < 1900:
        return 4
    elif t >= 1900 and t < 2300:
        return 5

def Addressing_Data(grades, v_info, records, student_info):
    records['time'] = list(map(to_datetime, records['created_at']))

    total_spent_time = [] # 總觀看時間
    total_watch_video_time = [] # 觀看的總影片時間

    length_l = list(v_info['meta'])# length of each lecture
    length_l = [int(i.split('n=>')[1].split('}')[0]) for i in length_l]

    sf = []
    period = [[0 for i in range(len(student_info['student_id']))] for i in range(6)]
    period_d = [[0 for i in range(len(student_info['student_id']))] for i in range(5)]
    forward_sec = []
    backward_sec =[]
    forward_times = []
    backward_times =[]
    pf = [] # pause freq
    backward_records = records[(records['start'] - records['end'] >= 5) & (records['playback_rate'] ==0)]
    forward_records = records[(records['start'] - records['end'] <= -5) & (records['playback_rate'] ==0)]
    pause_records = records[((records['start'] - records['end']).abs() < 5) & (records['playback_rate'] ==0)]

    j = 0
    for i in student_info['student_id']:
        watch_record_i = records[records['student_id'] == int(i)]
        
        ## 計算總觀看時間
        real_watch_i = watch_record_i[(watch_record_i['end'] > watch_record_i['start']) & (watch_record_i['playback_rate'] != 0)]# 抓出有在看的資料
        video_time_i = real_watch_i['end'] - real_watch_i['start']# 觀看紀錄 的 影片時間(end - start)
        total_watch_video_time.append(sum(video_time_i))
        total_spent_time.append(sum(video_time_i/real_watch_i['playback_rate']))

        ## 計算平均撥放次數
        wf = [] # each video watch freq
        for vi, vl in zip(v_info['video_id'], length_l):
            record_ji = watch_record_i[(watch_record_i['video_id'] == vi) & (watch_record_i['playback_rate'] != 0)]
            wf.append(sum(record_ji['end'] - record_ji['start'])/vl)
            
            if len(record_ji) != 0:
                first_watch = record_ji[record_ji['created_at'] == min(record_ji['created_at'])].head(1)['time']
                #print((first_watch))
                p = time_period(int(first_watch))
                #print('time: %d, p = %d' %(first_watch, p))
                period[p][j] += 1
                #print(min(record_ji['created_at']).index)
        j += 1
        sf.append(sum(wf)/len(wf))  

        # backwards
        backward_records_i = backward_records[backward_records['student_id'] == int(i)]
        backward_times.append(len(backward_records_i))
        backward_sec.append(sum(backward_records_i['start'] - backward_records_i['end']))
        
        # forwards
        forward_records_i = forward_records[forward_records['student_id'] == int(i)]
        forward_times.append(len(forward_records_i))
        forward_sec.append(sum(forward_records_i['end'] - forward_records_i['start']))

        # pause
        record_i = pause_records[(pause_records['student_id'] == int(i))]
        #display(record_i)
        pf.append(len(record_i))


    student_info['total_watch_time'] = total_spent_time
    student_info['watch_freq'] = sf
    student_info['backward_sec'] = backward_sec
    student_info['backward_times'] = backward_times
    student_info['forward_sec'] = forward_sec
    student_info['forward_times'] = forward_times
    student_info['pause_freq'] = pf

    for i in range(len(period)):
        student_info['period %d' %i] = period[i]
    
    student_info['most_watch_period'] = student_info[['period 0', 'period 1', 'period 2', 'period 3', 'period 4', 'period 5']].idxmax(axis = 1)
    student_info['watched_video'] = sum(np.array(period[i]) for i in range(6))

    ## 計算撥放速度 OK
    total_spent_time = [x if x!= 0 else 1 for x in total_spent_time]
    avg_playback_rate = [i/j for (i,j) in zip(total_watch_video_time, total_spent_time)]
    student_info['avg_playback_rate'] = avg_playback_rate

    return student_info
        


### 讀取資料

In [3]:
grades = pd.read_csv('../../data/DSAP107 grades.csv')
v_info = pd.read_excel('../../data/DSAP107 videos_sep.xlsx', sheet_name = 'video info')
records = pd.read_excel('../../data/DSAP107 videos_sep.xlsx', sheet_name = 'watching records')
student_id = pd.read_excel('../../data/DSAP107 videos_sep.xlsx', sheet_name = 'student id')


In [4]:
# 4/22以前
records = records[records['created_at'] < '2019-04-22 06:30:00 UTC']

## 要有的 column
- 其中、期末成績!
- 學生觀看時段
    - 23-3
    - 3-7
    - 7-11
    - 11-15
    - 15-19
    - 19-23
- 觀看總時間!
- 平均播放速度!
- 平均影片觀看次數!
- 暫停次數!

### 其中、期末成績

In [5]:
# 開一個 table 存等一下計算的資料
student_info = pd.DataFrame(data = grades.drop(0)['student_id'])

student_info['midterm'] = grades['Midterm (2042)']
student_info['final'] = grades['Final (2346)']

### 觀看時間、觀看速度、平均觀看次數、觀看時段（最早）

In [6]:
'''
### 觀看時間、觀看速度、平均觀看次數


# 計算總觀看時間
total_spent_time = [] # 總觀看時間
total_watch_video_time = [] # 觀看的總影片時間

for i in student_info['student_id']:
    watch_record_i = records[records['student_id'] == int(i)]
    # 抓出有在看的資料
    real_watch_i = watch_record_i[(watch_record_i['end'] > watch_record_i['start']) & (watch_record_i['playback_rate'] != 0)]
    video_time_i = real_watch_i['end'] - real_watch_i['start']# 觀看紀錄 的 影片時間(end - start)
    total_watch_video_time.append(sum(video_time_i))
    total_spent_time.append(sum(video_time_i/real_watch_i['playback_rate']))

student_info['total_watch_time'] = total_spent_time

# 計算撥放速度
total_spent_time = [x if x!= 0 else 1 for x in total_spent_time]
avg_playback_rate = [i/j for (i,j) in zip(total_watch_video_time, total_spent_time)]
student_info['avg_playback_rate'] = avg_playback_rate

'''

"\n### 觀看時間、觀看速度、平均觀看次數\n\n\n# 計算總觀看時間\ntotal_spent_time = [] # 總觀看時間\ntotal_watch_video_time = [] # 觀看的總影片時間\n\nfor i in student_info['student_id']:\n    watch_record_i = records[records['student_id'] == int(i)]\n    # 抓出有在看的資料\n    real_watch_i = watch_record_i[(watch_record_i['end'] > watch_record_i['start']) & (watch_record_i['playback_rate'] != 0)]\n    video_time_i = real_watch_i['end'] - real_watch_i['start']# 觀看紀錄 的 影片時間(end - start)\n    total_watch_video_time.append(sum(video_time_i))\n    total_spent_time.append(sum(video_time_i/real_watch_i['playback_rate']))\n\nstudent_info['total_watch_time'] = total_spent_time\n\n# 計算撥放速度\ntotal_spent_time = [x if x!= 0 else 1 for x in total_spent_time]\navg_playback_rate = [i/j for (i,j) in zip(total_watch_video_time, total_spent_time)]\nstudent_info['avg_playback_rate'] = avg_playback_rate\n\n"

In [7]:
'''
# 計算影片每部平均觀看次數 & 計算觀看時間
length_l = list(v_info['meta'])# length of each lecture
length_l = [int(i.split('n=>')[1].split('}')[0]) for i in length_l]

sf = []
period = [[0 for i in range(len(student_info['student_id']))] for i in range(6)]

i = 0
for j in student_info['student_id']:
    wf = [] # each video watch freq
    record_j = records[records['student_id'] == int(j)]
    
    for vi, vl in zip(v_info['video_id'], length_l):
        record_ji = record_j[(record_j['video_id'] == vi) & (record_j['playback_rate'] != 0)]
        wf.append(sum(record_ji['end'] - record_ji['start'])/vl)
        
        if len(record_ji) != 0:
            first_watch = record_ji[record_ji['created_at'] == min(record_ji['created_at'])].head(1)['time']
            #print((first_watch))
            p = time_period(int(first_watch))
            #print('time: %d, p = %d' %(first_watch, p))
            period[p][i] += 1
            #print(min(record_ji['created_at']).index)
    i += 1
    sf.append(sum(wf)/len(wf))  

student_info['watch_freq'] = sf
for i in range(len(period)):
    student_info['period %d' %i] = period[i]
student_info['watched_video'] = sum(np.array(period[i]) for i in range(6))
'''

"\n# 計算影片每部平均觀看次數 & 計算觀看時間\nlength_l = list(v_info['meta'])# length of each lecture\nlength_l = [int(i.split('n=>')[1].split('}')[0]) for i in length_l]\n\nsf = []\nperiod = [[0 for i in range(len(student_info['student_id']))] for i in range(6)]\n\ni = 0\nfor j in student_info['student_id']:\n    wf = [] # each video watch freq\n    record_j = records[records['student_id'] == int(j)]\n    \n    for vi, vl in zip(v_info['video_id'], length_l):\n        record_ji = record_j[(record_j['video_id'] == vi) & (record_j['playback_rate'] != 0)]\n        wf.append(sum(record_ji['end'] - record_ji['start'])/vl)\n        \n        if len(record_ji) != 0:\n            first_watch = record_ji[record_ji['created_at'] == min(record_ji['created_at'])].head(1)['time']\n            #print((first_watch))\n            p = time_period(int(first_watch))\n            #print('time: %d, p = %d' %(first_watch, p))\n            period[p][i] += 1\n            #print(min(record_ji['created_at']).index)\n    i += 1\

### 拖曳次數、拖曳時間長度 & 暫停次數

In [8]:
'''
forward_sec = []
backward_sec =[]
forward_times = []
backward_times =[]
pf = [] # pause freq

backward_records = records[(records['start'] - records['end'] >= 5) & (records['playback_rate'] ==0)]
forward_records = records[(records['start'] - records['end'] <= -5) & (records['playback_rate'] ==0)]
pause_records = records[((records['start'] - records['end']).abs() < 5) & (records['playback_rate'] ==0)]

for i in student_info['student_id']:
    # backwards
    backward_records_i = backward_records[backward_records['student_id'] == int(i)]
    backward_times.append(len(backward_records_i))
    backward_sec.append(sum(backward_records_i['start'] - backward_records_i['end']))
    
    # forwards
    forward_records_i = forward_records[forward_records['student_id'] == int(i)]
    forward_times.append(len(forward_records_i))
    forward_sec.append(sum(forward_records_i['end'] - forward_records_i['start']))

    # pause
    record_i = pause_records[(pause_records['student_id'] == int(i))]
    #display(record_i)
    pf.append(len(record_i))
    
student_info['backward_sec'] = backward_sec
student_info['backward_times'] = backward_times
student_info['forward_sec'] = forward_sec
student_info['forward_times'] = forward_times
student_info['pause_freq'] = pf
'''

"\nforward_sec = []\nbackward_sec =[]\nforward_times = []\nbackward_times =[]\npf = [] # pause freq\n\nbackward_records = records[(records['start'] - records['end'] >= 5) & (records['playback_rate'] ==0)]\nforward_records = records[(records['start'] - records['end'] <= -5) & (records['playback_rate'] ==0)]\npause_records = records[((records['start'] - records['end']).abs() < 5) & (records['playback_rate'] ==0)]\n\nfor i in student_info['student_id']:\n    # backwards\n    backward_records_i = backward_records[backward_records['student_id'] == int(i)]\n    backward_times.append(len(backward_records_i))\n    backward_sec.append(sum(backward_records_i['start'] - backward_records_i['end']))\n    \n    # forwards\n    forward_records_i = forward_records[forward_records['student_id'] == int(i)]\n    forward_times.append(len(forward_records_i))\n    forward_sec.append(sum(forward_records_i['end'] - forward_records_i['start']))\n\n    # pause\n    record_i = pause_records[(pause_records['stude

In [17]:
student_info = Addressing_Data(grades, v_info, records, student_info)
display(student_info.head())

,student_id,midterm,final,total_watch_time,watch_freq,backward_sec,backward_times,forward_sec,forward_times,pause_freq,period 0,period 1,period 2,period 3,period 4,period 5,watched_video,avg_playback_rate,most_watch_period
1,64,0,0.0,4385.133333,0.178199,0,0,0,0,0,0,0,0,0,10,2,12,1.494139,period 4
2,65,0,0.0,3572.000000,0.092675,88,5,442,9,4,4,0,0,0,0,1,5,1.000000,period 0
3,66,82,76.0,36625.000000,1.033947,6685,171,7912,248,22,36,0,7,6,0,3,52,1.000000,period 0
4,67,79,79.0,18789.383333,0.963616,6459,116,6784,161,6,3,3,9,33,1,1,50,1.847426,period 3
5,68,52,57.0,13719.766667,0.673371,3958,83,6892,171,35,2,0,3,29,8,0,42,1.742595,period 3


### Save data

In [10]:
student_info.to_excel('../../data/Semester weekly regression/DSAP107.xlsx', index = False)

### Regression

In [ ]:
import statsmodels.formula.api as smf
data_r = student_info.dropna(subset=['midterm'])
data_r = data_r[data_r['midterm'] != 0]
data_r['watched_video_l'] = np.log(data_r['watched_video'])
result = smf.ols('midterm ~ watched_video_l + total_watch_time', data = data_r).fit()
print(result.summary())

In [ ]:
plot_pd = data_r.copy()
plot_pd['prediction'] = result.predict()
n = len(plot_pd)
plot_pd = plot_pd.sort_values(by = ['midterm'])
plt.scatter(range(n), plot_pd['midterm'], label = 'Real Score')
plt.scatter(range(n), plot_pd['prediction'], label = 'prediction')
plt.legend()

In [ ]:
corr = data[["total_watch_time", 'avg_playback_rate','avg_freq','backward_sec', 
             'backward_times', 'forward_sec', 'forward_times']].corr()
sns.heatmap(corr, annot=True)